# Handling large data frame
Ran into difficulty processing data on AWS. Temporary solution was to split data locally 

In [93]:
import pandas as pd
#from io import StringIO
from gensim import corpora, models, similarities, matutils
#from sklearn import pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import svm
import re
import numpy as np

%matplotlib inline


#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

!ls project_4_kernel

Get_personality .ipynb comments.sample.csv
clean_labels2.csv      total.csv


# Importing data

In [3]:
import dask.dataframe as dd

In [4]:
#data is too large to fit into pd dataframe so fit into dd
df = dd.read_csv('10_12_all_comments.csv')

In [ ]:
df['comments'] = df['comments'].astype(str)

In [24]:
#count number of comments for each user 
com = df.comments.str.count(r', b"')

In [25]:
com

104      17
105      56
106       9
107      15
108     153
109      88
110      90
111       2
112       8
113      25
114      18
115      86
116      15
117     180
118    2744
119       8
120     328
121       0
122      84
123       0
124      63
125       8
126      96
127      94
128     250
129      11
130       4
131      12
132     209
133     710
       ... 
238     319
239      60
240      59
241     280
242       7
243      52
244     260
245     383
246     216
247      42
248     381
249     617
250     214
251     605
252      11
253     276
254     212
255      76
256      91
257    1306
258     330
259      54
260      29
261     881
262      74
263     520
264     216
265     114
266      83
267    1216
Name: comments, Length: 3212, dtype: int64

In [26]:
np.argmax(com)

299

In [ ]:
#drop largest
df.drop(df.index[299], inplace=True)

In [7]:
df.head()

,Unnamed: 0,user_orig,comments,personality_label
0,104,b'anynomous0000',"{b'I will ', b""That's fine, I don't."", b'What ...",INFJ
1,105,b'anyusernameideas',{b'my husband and i still meet our respective ...,INTJ
2,106,b'apa-theist',"{b""My doctor did the pregnancy tests with the ...",INTJ
3,107,b'aparaceu',"{b'Gender fluid restrooms, simply put, not saf...",INFP
4,108,b'apocryphalmaster',{b'Why exactly do they taste sweet? I imagined...,INTJ


# split large df into 2

In [8]:
df = np.array_split(df, 2)
df_split1 = pd.DataFrame(d1[0])
df_split2 = pd.DataFrame(df[1])

In [11]:
print(len(df_split2))
print(len(df_split1))

1601
1601


In [ ]:
#separates comments into separate columns
df1 = df_split1['comments'].str[1:-1].str.split(', b"', expand=True).astype(str)
df1 = df1.ix[:,0:249]

In [ ]:
df2 = df_split2['comments'].str[1:-1].str.split(', b"', expand=True).astype(str)
df2 = df2.ix[:,0:249]

In [14]:
df1 = df1.replace(r'None ','No one',regex=True).replace(r' None','No one', regex = True)
df1 = df1.replace(r'None',np.nan,regex=True)

In [25]:
df2 = df2.replace(r'None ','No one',regex=True).replace(r' None','No one', regex = True)
df2 = df2.replace(r'None',np.nan,regex=True)

In [15]:
def new_df(old):
    """merges every 5 columns"""
    dat1 = pd.DataFrame()
    m = 0
    n = 5
    while n <= 250:
        dat1["col_"+str(n)] = old.iloc[:, m:n].fillna(' ').apply(lambda x: ','.join(x), axis = 1)
        m += 5
        n += 5
    dat1 = dat1.replace(' , , , , ', np.NaN)
    return dat1

In [26]:
df1 = pd.DataFrame(new_df(df1))
df2 = pd.DataFrame(new_df(df2))

In [27]:
df2.head()

,col_5,col_10,col_15,col_20,col_25,col_30,col_35,col_40,col_45,col_50,...,col_205,col_210,col_215,col_220,col_225,col_230,col_235,col_240,col_245,col_250
1601,"b'I retook it today and got 60% ISFP.', b'For ...",Ha! This is so true of me. Even the best story...,I definitely know what you mean here. I love m...,"Officially, that's exactly what it is. 1980-20...","Give up something. Chocolate, TV, coffee...any...",I worked in a career field I didn't really lik...,Of course they don't have to be philanthropic....,He's a recovered alcoholic. He's been an absol...,Can you speak to a lawyer maybe? There is abso...,People talk about our oil consumption in terms...,...,How can you handle the wrong your? This person...,I think I was swapping between zooming in with...,Make sure you're in the right career. This is ...,To anyone else who replies...I appreciate thes...,NaN,NaN,NaN,NaN,NaN,NaN
1602,b'Is that like how google plus ruined facebook...,You have no idea what you're talking about and...,They're losing a pick to the Sixers regardless...,What's your guess at a b/o price or range if t...,Didn't Ainge also try to trade 4 firsts for Fr...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1603,"b""You should go on the hogwarts castle ride an...",I grew up seeing this with my Dad and Stepmom....,Thanks for the reply. I've done better on glu...,I say therapist. If you haven't tried a dieti...,"Giraffes but I really don't know why."", b'I on...",I was really disappointed Jarvis wasn't an opt...,Build a hydroelectric plant over the river por...,Yup watched a lady with 3 cart fulls of grocer...,I'm not good at other types but I'd guess one ...,Chris Pratt is hard to beat for me right now. ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1604,b'Have you figured out the reasons you cheated...,You really can't judge your recovery time by y...,I think it's to justify cheating too tbh. Sayi...,Are there any legal repercussions for us? I'm ...,I feel your pain. The first I heard about my S...,What you lose does tend to come back most of t...,What you lose does tend to come back most of t...,What you lose does tend to come back most of t...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1605,"b""I'm an atheist now and went to Liberty as we...",I'm so much like this. Recently went through a...,"Yay for introverted, liberal, feminist, atheis...","Yay for introverted, liberal, feminist, atheis...",I have never despised a character so much in a...,"Ha, well, we actually met on tinder. We immedi...",Has not stopped me from recommending it. Absol...,"The lump is on top of my breast, not in it rea...",It's been years since I left Buffalo and I sti...,I'm in the same spot. It's getting really diff...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_split2.head()

,Unnamed: 0,user_orig,comments,personality_label
1601,90,b'malachai926',"{b'I retook it today and got 60% ISFP.', b'For...",ISFP
1602,91,b'malepussycat',{b'Is that like how google plus ruined faceboo...,ENFP
1603,92,b'mallutts',"{b""You should go on the hogwarts castle ride a...",ISTJ
1604,93,b'mancyelle',{b'Have you figured out the reasons you cheate...,ESFJ
1605,94,b'mandakey',"{b""I'm an atheist now and went to Liberty as w...",INTJ


In [29]:
df_split1.drop('comments', inplace=True, axis=1)
df_split2.drop('comments', inplace=True, axis=1)

In [34]:
df_split1.head()

,user_orig,personality_label
0,b'-1mar7-',ISTP
1,b'-Afrodisiac-',ENTP
2,b'-Avatar-Korra-',INTP
3,b'-B1ackRibbon-',ENTP
4,b'-JungleB-',INFP


In [30]:
df1_results = pd.concat([df_split1, df1], axis=1)
df2_results = pd.concat([df_split2, df2], axis=1)

In [20]:
df1_results.drop('comments', inplace=True, axis=1)
df2_results.drop('comments', inplace=True, axis=1)

In [29]:
#rejoin frames

frames = [df1_results, df2_results]
df = pd.concat(frames)

In [30]:
df.head()

,Unnamed: 0,Unnamed: 0.1,col_10,col_100,col_105,col_110,col_115,col_120,col_125,col_130,...,col_60,col_65,col_70,col_75,col_80,col_85,col_90,col_95,personality_label,user_orig
0,0,NaN,I don't. My chair fits on the right side of th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISTP,b'-1mar7-'
1,1,NaN,How's that Dash Charging? And have you had any...,"Sure, but I've met just as many overweight MRA...","Perhaps unhealthy IxFP's, but most FP's that I...",As someone who's had a... less than satisfacto...,"I've never been more hopeful, lol."", b'I remem...",No idea why you're getting down voted -- Alexa...,"Thank you, I appreciate it! You're a fantastic...","I haven't done CBT and tbh, I'm kind of afraid...",...,yeah! i've heard a lot of stories about people...,"Oh, no. What's not to like? I haven't had any ...",You bring up an excellent point -- speech Trum...,"Hey, I just wanted to let you know there's act...","Don't worry about it, he's used to being forgo...","Where do you go to school, if you don't mind?""...",Most people under the age of 35 couldn't eithe...,INFJ's are also known for inspiring masses. EN...,ENTP,b'-Afrodisiac-'
2,2,NaN,"Sorry, I just clicked on the comments of one o...",I don't think they would ever make Marquee eve...,On a regular team? No. GG is bad right now \n\...,&gt;Your character is self-important and bigot...,I posted a serious one. Although the titles of...,"Man, I have stuff I want to rant about or just...","Yeah, it is because they weren't all together....","Yeah, but odds are that you aren't too differe...",...,Didn't you hear her new catch phrase? SUCK A D...,What are you complaining about? Before this we...,&gt; I then saw this trick on ifuuny right aft...,Yep. I often blackmail my targets to force the...,"It's not for Wizard101?"", b'I mean, Marco is t...","But the Jedi didn't trust him, they disliked h...","*IT'S YOUR NEW FAVORITE SHOW*"", b'These girls ...",It depends on my classes. We just switched to ...,INTP,b'-Avatar-Korra-'
3,3,NaN,"Aww, well I'm still looking forward to hearing...","ik lol, they're a hacktivist group that suppor...","Damn, so much good news today!\nBig thanks to ...","I'm down to be tied down."",I don't like to tre...",Yeah this is pretty much how it is... What's g...,"1) Yes I do, I provided a link to make it easi...",NaN,NaN,...,In my experience most psychologists fully supp...,Oh crap I didn't know there was a taiHEN githu...,"Honestly, I think you'd have a better time usi...",Oooohhh this should be interesting! I'm very s...,KMFDM is still pretty political and edgy but i...,"you don't have to do load core."",Also use an o...","* You're fucked. XD"", b'Yeah, I made it so tha...",Can't you just rename the Katamari vpk file to...,ENTP,b'-B1ackRibbon-'
4,4,NaN,Here's my only NSMBU level that's up to my cur...,Certain pokemon now have moves they will be gu...,My brother is an ENFP and he's one of the most...,"So, I'm not sure if this is just on my end or ...",This is should a very good point. I would have...,"Person here with asperger's syndrome, another ...",Looks like an awesome idea for a series of pos...,I just realized I won't be able to play it unt...,...,Oh hey! My first time getting in the top 3! Th...,"You're getting arm-arrested for that joke. "",Y...",I remember when this was first posted! Still a...,"On top of that, the rise of the internet has a...","On top of that, the rise of the internet has a...",That actually sounds really good. I'd buy that...,Congrat(toffie)! Glad you finally got a win! \...,"I know that, I moreso mean it feels bad that n...",INFP,b'-JungleB-'


In [31]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)

In [32]:
df_multi = df.set_index(['personality_label', 'user_orig']).stack()

In [34]:
df_stacked = df_multi.reset_index().rename(columns={0:'comment'})

In [35]:
df_stacked.head()

,personality_label,user_orig,level_2,comment
0,ISTP,b'-1mar7-',col_10,I don't. My chair fits on the right side of th...
1,ISTP,b'-1mar7-',col_5,"b""I'm a student but - \nHad: worked at a dog b..."
2,ENTP,b'-Afrodisiac-',col_10,How's that Dash Charging? And have you had any...
3,ENTP,b'-Afrodisiac-',col_100,"Sure, but I've met just as many overweight MRA..."
4,ENTP,b'-Afrodisiac-',col_105,"Perhaps unhealthy IxFP's, but most FP's that I..."


In [36]:
#lil mislabeled straggler 
df_stacked['personality_label'].replace("INSP", 'INFP',inplace=True)

In [39]:
df = copy.deepcopy(df_stacked)

In [51]:
ie = []
ns = []
ft = []
pj = []
for row in df['personality_label']:
    ie.append(row[0])
    ns.append(row[1])
    ft.append(row[2])
    pj.append(row[3])

df['EI'] = ie 
df['NS'] = ns
df['FT'] = ft
df['PJ'] = pj

In [41]:
df.to_csv('9_15_comments_file.csv')